<a href="https://colab.research.google.com/github/Benjamindavid03/MachineLearningLab/blob/main/Root_Node_Attribute_Selection_for_Decision_Tree_Info_Gain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Root Node Attribute Selection for Decision Tree using Information Gain

<b>Algorithm</b><br/>
Step-1 : Keep the original set S as the initial node.

Step-2 : On each iteration of the algorithm, iterate through the very unused attribute of the set S and 

Step-3: calculates Entropy(H) for the attribute.

Step-4: calculate Information gain(IG) of the attribute.

Step-5: Select the attribute which has the Largest Information gain and that is selected as the required root node.

## Step 1 : Let's first import the dataset from the Cloud.


In [ ]:
import pandas as pd
import numpy as np
import io
import requests

url_name = 'https://raw.githubusercontent.com/akmand/datasets/master/FMLPDA_Table4_3.csv'
url_content = requests.get(url_name, verify=False).content
fruits = pd.read_csv(io.StringIO(url_content.decode('utf-8')))
fruits

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,stream,slope,elevation,vegetation
0,False,steep,high,chapparal
1,True,moderate,low,riparian
2,True,steep,medium,riparian
3,False,steep,medium,chapparal
4,False,flat,high,conifer
5,True,steep,highest,conifer
6,True,steep,high,chapparal


For convenience, we define a function called compute_impurity() that calculates impurity of a feature using either entropy or gini index.

In [ ]:
def compute_impurity(feature, impurity_criterion):
    """
    This function calculates impurity of a feature.
    Supported impurity criteria: 'entropy', 'gini'
    input: feature (this needs to be a Pandas series)
    output: feature impurity
    """
    probs = feature.value_counts(normalize=True)
    
    if impurity_criterion == 'entropy':
        impurity = -1 * np.sum(np.log2(probs) * probs)
    elif impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))


# let's do two quick examples.
print('impurity using entropy:', compute_impurity(fruits, 'entropy'))
print('impurity using gini index:', compute_impurity(fruits, 'gini'))

impurity using entropy: 2.807
impurity using gini index: 0.857


A sample way to compute the entropy of the vegetation feature from the dataset

In [ ]:
target_entropy = compute_impurity(fruits['vegetation'], 'entropy')
target_entropy

1.557

## Step 2 : Compute Entropy

Let's compute the information gain for splitting based on a descriptive feature to figure out the best feature to split on. For this task, we do the following:

Compute impurity of the target feature (using either entropy or gini index).
Partition the dataset based on unique values of the descriptive feature.
Compute impurity for each partition.
Compute the remaining impurity as the weighted sum of impurity of each partition.
Compute the information gain as the difference between the impurity of the target feature and the remaining impurity.
We will define another function to achieve this, called comp_feature_information_gain().

As an example, let's have a look at the levels of the "elevation" feature.

In [ ]:
fruits['elevation'].value_counts()

high       3
medium     2
low        1
highest    1
Name: elevation, dtype: int64

In [ ]:
fruits['slope'].value_counts()

steep       5
moderate    1
flat        1
Name: slope, dtype: int64

Let's see how the partitions look like for this feature and what the corresponding calculations are using the entropy split criterion (entropy).

In [ ]:
for level in fruits['elevation'].unique(): #Series.unique() In this example, unique() method is used to know all type of unique values in Team column.
    print('level name:', level)
    df_feature_level = fruits[fruits['elevation'] == level]
    print('corresponding data partition:')
    print(df_feature_level)
    print('partition target feature impurity:', compute_impurity(df_feature_level['vegetation'], 'entropy'))
    print('partition weight:', str(len(df_feature_level)) + '/' + str(len(fruits)))
    print('====================')


level name: high
corresponding data partition:
   stream  slope elevation vegetation
0   False  steep      high  chapparal
4   False   flat      high    conifer
6    True  steep      high  chapparal
partition target feature impurity: 0.918
partition weight: 3/7
level name: low
corresponding data partition:
   stream     slope elevation vegetation
1    True  moderate       low   riparian
partition target feature impurity: -0.0
partition weight: 1/7
level name: medium
corresponding data partition:
   stream  slope elevation vegetation
2    True  steep    medium   riparian
3   False  steep    medium  chapparal
partition target feature impurity: 1.0
partition weight: 2/7
level name: highest
corresponding data partition:
   stream  slope elevation vegetation
5    True  steep   highest    conifer
partition target feature impurity: -0.0
partition weight: 1/7


## Step 3 : Computing Information Gain
The idea here is that, for each one of the 4 data partitions above, we compute their impurity with respect to the target feature as a stand-alone dataset.

We weigh these impurities with the relative number of observations in each partition. The relative number of observations is calculated as the number of observations in the partition divided by the total number of observations in the entire dataset. For instance, the weight of the first partition is 3/7.
We add up these weighted impurities and call it the remaining impurity for this feature.
For instance, remaining impurity as measured by entropy for the elevation feature is <p>0.918 x (3/7) + 1.0 x (2/7) = 0.679.</p>

Information gain is then calculated as <p style='background-color:red'> 1.557 - 0.679 = 0.878. </p>

Now we are ready to define our function. There is a bit of coding in here, but we can assure you that trying to figure out how things work in here will be rewarding to improve your Python programming skills.

In [ ]:
def comp_feature_information_gain(df, target, descriptive_feature, split_criterion):
    """
    This function calculates information gain for splitting on 
    a particular descriptive feature for a given dataset
    and a given impurity criteria.
    Supported split criterion: 'entropy', 'gini'
    """
    
    print('target feature:', target)
    print('descriptive_feature:', descriptive_feature)
    print('split criterion:', split_criterion)
            
    target_entropy = compute_impurity(df[target], split_criterion)

    # we define two lists below:
    # entropy_list to store the entropy of each partition
    # weight_list to store the relative number of observations in each partition
    entropy_list = list()
    weight_list = list()
    
    # loop over each level of the descriptive feature
    # to partition the dataset with respect to that level
    # and compute the entropy and the weight of the level's partition
    for level in df[descriptive_feature].unique():
        df_feature_level = df[df[descriptive_feature] == level]
        entropy_level = compute_impurity(df_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))

    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)

    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity)
    
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', information_gain)
    
    print('====================')

    return(information_gain)

Now that our function has been defined, we will call it for each descriptive feature in the dataset. First let's call it using the entropy split criteria.

## Step 4 : Find the best feature with the highest impurity and place it at the root node.

In [ ]:
split_criterion = 'entropy'
for feature in fruits.drop(columns='vegetation').columns:
    feature_info_gain = comp_feature_information_gain(fruits, 'vegetation', feature, split_criterion)

target feature: vegetation
descriptive_feature: stream
split criterion: entropy
impurity of partitions: [0.918, 1.5]
weights of partitions: [0.429, 0.571]
remaining impurity: 1.250322
information gain: 0.306678
target feature: vegetation
descriptive_feature: slope
split criterion: entropy
impurity of partitions: [1.371, -0.0, -0.0]
weights of partitions: [0.714, 0.143, 0.143]
remaining impurity: 0.9788939999999999
information gain: 0.578106
target feature: vegetation
descriptive_feature: elevation
split criterion: entropy
impurity of partitions: [0.918, -0.0, 1.0, -0.0]
weights of partitions: [0.429, 0.143, 0.286, 0.143]
remaining impurity: 0.6798219999999999
information gain: 0.877178


Step : Now let's call it using the gini index split criteria. (NOT FOR EXAM)

In [ ]:
split_criteria = 'gini'
for feature in fruits.drop(columns='vegetation').columns:
    feature_info_gain = comp_feature_information_gain(fruits, 'vegetation', feature, split_criteria)

target feature: vegetation
descriptive_feature: stream
split criterion: gini
impurity of partitions: [0.444, 0.625]
weights of partitions: [0.429, 0.571]
remaining impurity: 0.5473509999999999
information gain: 0.1056490000000001
target feature: vegetation
descriptive_feature: slope
split criterion: gini
impurity of partitions: [0.56, 0.0, 0.0]
weights of partitions: [0.714, 0.143, 0.143]
remaining impurity: 0.39984000000000003
information gain: 0.25316
target feature: vegetation
descriptive_feature: elevation
split criterion: gini
impurity of partitions: [0.444, 0.0, 0.5, 0.0]
weights of partitions: [0.429, 0.143, 0.286, 0.143]
remaining impurity: 0.333476
information gain: 0.31952400000000003


We observe that, with both the entropy and gini index split criteria, the highest information gain occurs with the "elevation" feature.

This is the for the split at the root node of the corresponding decision tree. In subsequent splits, the above procedure is repeated with the subset of the entire dataset in the current branch until the termination condition is reached.



References

1. https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
2. https://en.wikipedia.org/wiki/Decision_tree_learning#Information_Gain
3. http://christianherta.de/lehre/dataScience/machineLearning/decision-trees.php
4. https://www.featureranking.com/tutorials/machine-learning-tutorials/information-gain-computation/
